<a href="https://colab.research.google.com/github/dmarchignoli/haystack-rag/blob/main/ingest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!rm -rf /content/sample_data
!(cd /content/haystack-rag/ && git pull) || git clone https://github.com/dmarchignoli/haystack-rag.git /content/haystack-rag

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 5 (delta 2), reused 5 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 525 bytes | 262.00 KiB/s, done.
From https://github.com/dmarchignoli/haystack-rag
   a7f5384..19b06af  main       -> origin/main
Updating a7f5384..19b06af
Fast-forward
 src/haystack_rag/ingest.py | 24 ++----------------------
 1 file changed, 2 insertions(+), 22 deletions(-)


In [3]:
%cd /content/haystack-rag/
import sys, os
sys.path.append(os.path.join(os.getcwd(), 'src'))

/content/haystack-rag


In [4]:
!pip install 'haystack-ai>=2.7.0' 'sentence-transformers>=3.3.1' 'unstructured-fileconverter-haystack>=0.4.1'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.4/391.4 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 26.1 MB/s eta 0:00:00


In [15]:
import haystack_rag.ingest as ingest

lib_urls = ingest.load_library_urls()
print(lib_urls)

['https://www.bancaditalia.it/pubblicazioni/guide-bi/guida-conto-corrente/Le_guide_della_Banca_d_Italia_Il_conto_corrente_in_parole_semplici.pdf', 'https://www.bancaditalia.it/pubblicazioni/guide-bi/guida-mutuo/Le-guide-della-Banca-d-Italia_Comprare-una-casa_Il-mutuo-ipotecario-in-parole-semplici.pdf', 'https://www.bancaditalia.it/pubblicazioni/guide-bi/guida-credito-consumatori/Le-guide-della-Banca-d-Italia_Il-credito-ai-consumatori-in-parole-semplici.pdf', 'https://www.bancaditalia.it/pubblicazioni/guide-bi/guida-centrale/Guida-centrale-rischi.pdf', 'https://www.bancaditalia.it/pubblicazioni/guide-bi/guida-pagamenti-comm-elettronico/guide-BI-i-pagamenti-nel-commercio-elettronico_ITA.pdf']


In [16]:
from haystack.components.fetchers import LinkContentFetcher

fetcher = LinkContentFetcher()

r = fetcher.run(urls=lib_urls)
docscontent = r['streams']
print(type(docscontent), len(docscontent), type(docscontent[0]))

docscontent[0].meta

<class 'list'> 5 <class 'haystack.dataclasses.byte_stream.ByteStream'>


{'content_type': 'application/pdf',
 'url': 'https://www.bancaditalia.it/pubblicazioni/guide-bi/guida-conto-corrente/Le_guide_della_Banca_d_Italia_Il_conto_corrente_in_parole_semplici.pdf'}

In [18]:
from haystack_rag.ingest import  ByteStreamMaterializer

materializer = ByteStreamMaterializer()
r = materializer.run(docscontent)

docs_paths = r['paths']
print(docs_paths)


[PosixPath('/root/.cache/haystack/Le_guide_della_Banca_d_Italia_Il_conto_corrente_in_parole_semplici.pdf'), PosixPath('/root/.cache/haystack/Le-guide-della-Banca-d-Italia_Comprare-una-casa_Il-mutuo-ipotecario-in-parole-semplici.pdf'), PosixPath('/root/.cache/haystack/Le-guide-della-Banca-d-Italia_Il-credito-ai-consumatori-in-parole-semplici.pdf'), PosixPath('/root/.cache/haystack/Guida-centrale-rischi.pdf'), PosixPath('/root/.cache/haystack/guide-BI-i-pagamenti-nel-commercio-elettronico_ITA.pdf')]


In [24]:
import os
from haystack_integrations.components.converters.unstructured import UnstructuredFileConverter
from google.colab import userdata
from haystack.utils import Secret

converter = UnstructuredFileConverter(
    api_url='https://unstructured-api-826421350323.us-central1.run.app/',
    api_key=Secret.from_token(userdata.get('UNSTRUCTURED_API_KEY'))
)
r = converter.run(docs_paths)
documents = r['documents']
print(type(documents), len(documents), type(documents[0]))

Converting files to Haystack Documents: 0it [00:00, ?it/s]WARNING:unstructured-client:'split_pdf_cache_tmp_data' does not exist. Using default value '/tmp'.
Converting files to Haystack Documents: 1it [04:09, 249.27s/it]WARNING:unstructured-client:'split_pdf_cache_tmp_data' does not exist. Using default value '/tmp'.
Converting files to Haystack Documents: 2it [05:57, 166.10s/it]WARNING:unstructured-client:'split_pdf_cache_tmp_data' does not exist. Using default value '/tmp'.
Converting files to Haystack Documents: 3it [07:30, 132.93s/it]WARNING:unstructured-client:'split_pdf_cache_tmp_data' does not exist. Using default value '/tmp'.
Converting files to Haystack Documents: 4it [08:28, 103.13s/it]WARNING:unstructured-client:'split_pdf_cache_tmp_data' does not exist. Using default value '/tmp'.
Converting files to Haystack Documents: 5it [10:14, 122.81s/it]


KeyError: 0

In [39]:
import pickle

for d in documents:
  op = d.meta['file_path']+".pickle"
  with open(op, "wb") as f:
    pickle.dump(d, f)

#documents = documents['documents']
print(type(documents), len(documents), type(documents[0]))
print(dir(documents[0]))
print(documents[0])

<class 'list'> 5 <class 'haystack.dataclasses.document.Document'>
['__annotations__', '__class__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__post_init__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_create_id', 'blob', 'content', 'content_type', 'dataframe', 'embedding', 'from_dict', 'id', 'meta', 'score', 'sparse_embedding', 'to_dict']
Document(id=a34c49a0788b316c26b67a50dd5c73c4c3aab32e97b28799de9a97719da162f7, content: '

LE GUIDE DELLA BANCA D’ITALIA

IL CONTO CORRENTE in parole semplici

La La SCELTA  SCELTA e i COST...', meta: {'file_path': '/root/.cache/haystack/Le_guide_della_Banca_d_Italia_Il_conto_corrente_in_parole_semplici.pdf'})


In [41]:
!pip install --upgrade google-cloud-storage

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 2.8.0
    Uninstalling google-cloud-storage-2.8.0:
      Successfully uninstalled google-cloud-storage-2.8.0


In [43]:
from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'cellular-ring-443811-t2'
from google.cloud import storage

# Instantiates a client
storage_client = storage.Client(project=project_id)

# The name for the new bucket
bucket_name = 'haystack-docs-826421350323'

# Creates the new bucket
bucket = storage_client.bucket(bucket_name)

#print("Bucket created successfully")


Conflict: 409 POST https://storage.googleapis.com/storage/v1/b?project=cellular-ring-443811-t2&prettyPrint=false: The requested bucket name is not available. The bucket namespace is shared by all users of the system. Please select a different name and try again.

In [ ]:
from haystack import Pipeline

pipeline = Pipeline()

pipeline.add_component(instance=fetcher, name="fetcher")

pipeline.add_component(instance=materializer, name="materializer")
pipeline.connect("fetcher", "materializer")

pipeline.warm_up()